In [45]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Embedding, Concatenate, Lambda, Layer
from tools.custom_layers import SqueezeLayer

In [78]:
# === Load Dataset ===
data = np.load("dataset/dataset-v0-1.npz")
X = data["X"]        # shape (N, 60, 68)
y = data["y"]        # shape (N,)
angles = data["angles"]  # shape (N,)

In [79]:
# === Define model parameters ===
sequence_length = X.shape[1]        # 60
feature_dim = X.shape[2]            # 68
num_classes = len(np.unique(y))     # e.g., 7 throw types
num_angles = len(np.unique(angles)) # e.g., 3 angles (side/front/diagonal)
angle_embed_dim = 4                 # Size of angle embedding vector

In [80]:
# === Input 1: Pose Sequence ===
pose_input = Input(shape=(sequence_length, feature_dim), name="pose_input")
x = LSTM(64, return_sequences=False)(pose_input)  # Output shape: (batch, 64)

In [81]:
# === Input 2: Angle Class Index ===
angle_input = Input(shape=(1,), dtype="int32", name="angle_input")
angle_embed = Embedding(input_dim=num_angles, output_dim=angle_embed_dim, name="angle_embedding")(angle_input)
angle_embed = SqueezeLayer(axis=1, name="squeeze_angle_dim")(angle_embed)

In [82]:
# === Combine Inputs ===
combined = Concatenate(name="concat_pose_angle")([x, angle_embed])
combined = Dropout(0.3)(combined)
output = Dense(num_classes, activation="softmax", name="classification_output")(combined)

In [83]:
model = Model(inputs=[pose_input, angle_input], outputs=output)

In [84]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ angle_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pose_input          │ (None, 60, 68)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ angle_embedding     │ (None, 1, 4)      │         16 │ angle_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 64)        │     34,048 │ pose_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ squeeze_angle_dim   │ (None, 4)         │          0 │ angle_embedding[… │
│ (SqueezeLayer)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat_pose_angle   │ (None, 68)        │          0 │ lstm_4[0][0],     │
│ (Concatenate)       │                   │            │ squeeze_angle_di… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 68)        │          0 │ concat_pose_angl… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ classification_out… │ (None, 6)         │        414 │ dropout_3[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 34,478 (134.68 KB)

 Trainable params: 34,478 (134.68 KB)

 Non-trainable params: 0 (0.00 B)

In [85]:
from sklearn.model_selection import train_test_split

In [86]:
# Reshape angle input for training
angle_input = angles.reshape(-1, 1)

# Split
X_train, X_test, y_train, y_test, a_train, a_test = train_test_split(
    X, y, angle_input, test_size=0.2, random_state=42, stratify=y
)

In [87]:
#NEWWWWWW
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# compute class weights
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# convert to dictionary format
class_weight_dict = {i: weight for i, weight in enumerate(class_weights_array)}

print("Class weights:", class_weight_dict)

Class weights: {0: 1.0266666666666666, 1: 1.2833333333333334, 2: 1.6041666666666667, 3: 0.6936936936936937, 4: 0.7129629629629629, 5: 1.2833333333333334}


In [88]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

In [89]:
# Train
history = model.fit(
    x={"pose_input": X_train, "angle_input": a_train},
    y=y_train,
    validation_data=(
        {"pose_input": X_test, "angle_input": a_test},
        y_test
    ),
    batch_size=16,
    epochs=2000, # Was originally 20 (15 also worked)
    callbacks=[early_stop], #added in to see
    class_weight=class_weight_dict #NEWWWWW
)

Epoch 1/2000
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.1703 - loss: 1.8440 - val_accuracy: 0.3333 - val_loss: 1.6245
Epoch 2/2000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4132 - loss: 1.5377 - val_accuracy: 0.3590 - val_loss: 1.4733
Epoch 3/2000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5145 - loss: 1.3515 - val_accuracy: 0.5897 - val_loss: 1.3096
Epoch 4/2000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5924 - loss: 1.1518 - val_accuracy: 0.6410 - val_loss: 1.1538
Epoch 5/2000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6868 - loss: 0.9416 - val_accuracy: 0.6923 - val_loss: 0.9566
Epoch 6/2000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7109 - loss: 0.7501 - val_accuracy: 0.7436 - val_loss: 0.7911
Epoch 7/2000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7458 - loss: 0.6326 - val_accuracy: 0.8205 - val_loss: 0.6996
Epoch 8/2000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8150 - loss: 0.5691 - val_accu

In [90]:
preds = model.predict({
    "pose_input": X_test[:1],
    "angle_input": a_test[:1]
})
predicted_class = np.argmax(preds, axis=-1)
print("Predicted throw class:", predicted_class[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Predicted throw class: 4


In [91]:
model.save("models/throw_detection_v0-1.keras")
